In [1]:
# Install dependencies
!pip install python-dotenv
!pip install openai
!pip install openpyxl
!pip uninstall openai

In [1]:
import os

import dotenv
import json
import re
import openai
import time
import sqlite3
import json

from dotenv import load_dotenv
load_dotenv()

from threading import Thread

openai.api_key = ('') # Set api key

MAX_FAILURES = 4

scenarios = dict({ 
    "Berk29": ((400, 400), (750, 400)),
    "Barc2": ((400, 400), (750, 375)),
    "Berk23": ((800, 200), (0, 0)),
    "Barc8": ((300, 600), (700, 500)),
    "Berk15": ((200, 700), (600, 600)),
    "Berk26":((0, 800), (400, 400)),
    "Gabe2" : ((200, 800), (600, 700)), # Two more scenarios
    "Gabe3" : ((0, 800), (410, 390))
})

def create_prompt(scenario, personality, author):
    left_a, left_b = scenario[0]
    right_a, right_b = scenario[1]
    return f"""You are the fictional {personality}. 
- Your task is to make a decision in a game that determines a payment to you and Person A.
- Person A has no choice in this game. You and Person A do not meet and will remain anonymous.
- Make your decision using what you know of {personality} from the original {author} work and all subsequent commentary.
- In this game you have two options:

Option Left: Person A receives {left_a}, and you receive {left_b}, 
Option Right: Person A receives {right_a}, and you receive {right_b}, 

What do you choose, with one word [Left, Right]?

Output two columns: 1) the choice you made [left or right], 2) the amount you receive. Do not output anything else.  An example of the format of the output is:  Right, 700
"""



def get_decision(scenario, personality, author, scenario_name, model):
    prompt = create_prompt(scenario, personality, author)
    failure_count = 0
    message_history = []
    message_history.append({"role":"user", "content": prompt})
    # Get response
    completion = openai.ChatCompletion.create(
        model='gpt-4',
        messages=message_history,
    )
    reply_content1 = completion.choices[0].message.content

    return dict({"raw_reply": reply_content1,
                 "choice": "Left" if "left" in reply_content1.lower() else "Right", 
                 "scenario": scenario,
                 "personality":personality, # Character
                 "model":model,
                 "scenario_name":scenario_name,
                 "prompt":prompt
                })


In [2]:
from time import sleep

class GetDictatorChoiceThread(Thread):

    def __init__(self, scenario, personality, author, scenario_name, model):
        super().__init__()
        self.scenario = scenario
        self.personality = personality
        self.scenario_name = scenario_name
        self.model = model
        self.author = author

    def run(self):
        self.decision = get_decision(scenario = self.scenario, personality = self.personality, author = self.author, scenario_name = self.scenario_name, model = self.model)

models_described = dict({
    "gpt-4": ("GPT-4 is more creative and collaborative than ever before. It can generate, edit, and iterate with users on creative and technical writing tasks, such as composing songs, writing screenplays, or learning a user’s writing style.")})

models = models_described.keys()
# List of personalities
personalities = [
   'Theo Decker'
     ]

# List of authors corresponding to personalities
authors = [
    'Donna Tartt'
]
 
threads = []
for model in models:
    for i, personality in enumerate(personalities):
        for scenario_name, scenario in scenarios.items():
            thread = GetDictatorChoiceThread(scenario = scenario, personality = personality, author=authors[i], scenario_name = scenario_name, model = model)
            thread.start()
            threads.append(thread)
    
observations = []
for thread in threads:
    error = 0
    while error < 5:
        try:
            thread.join()
            thread.run()
            observations.append((thread.decision, thread.author))
            break
        except Exception as e:
            error+=1
            print(e)
    

    

    




[({'raw_reply': 'Right, 400', 'choice': 'Right', 'scenario': ((400, 400), (750, 400)), 'personality': 'Theo Decker', 'model': 'gpt-4', 'scenario_name': 'Berk29', 'prompt': 'You are the fictional Theo Decker. \n- Your task is to make a decision in a game that determines a payment to you and Person A.\n- Person A has no choice in this game. You and Person A do not meet and will remain anonymous.\n- Make your decision using what you know of Theo Decker from the original Donna Tartt work and all subsequent commentary.\n- In this game you have two options:\n\nOption Left: Person A receives 400, and you receive 400, \nOption Right: Person A receives 750, and you receive 400, \n\nWhat do you choose, with one word [Left, Right]?\n\nOutput two columns: 1) the choice you made [left or right], 2) the amount you receive. Do not output anything else.  An example of the format of the output is:  Right, 700\n'}, 'Donna Tartt')]
[({'raw_reply': 'Right, 400', 'choice': 'Right', 'scenario': ((400, 400),

That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 67df15909e11ba8009af66073b7bff9f in your message.)
[({'raw_reply': 'Right, 400', 'choice': 'Right', 'scenario': ((400, 400), (750, 400)), 'personality': 'Theo Decker', 'model': 'gpt-4', 'scenario_name': 'Berk29', 'prompt': 'You are the fictional Theo Decker. \n- Your task is to make a decision in a game that determines a payment to you and Person A.\n- Person A has no choice in this game. You and Person A do not meet and will remain anonymous.\n- Make your decision using what you know of Theo Decker from the original Donna Tartt work and all subsequent commentary.\n- In this game you have two options:\n\nOption Left: Person A receives 400, and you receive 400, \nOption Right: Person A receives 750, and you receive 400, \n\nWhat do you choose, with one word [Left, Right]?\n\nOutput two columns: 

[({'raw_reply': 'Right, 400', 'choice': 'Right', 'scenario': ((400, 400), (750, 400)), 'personality': 'Theo Decker', 'model': 'gpt-4', 'scenario_name': 'Berk29', 'prompt': 'You are the fictional Theo Decker. \n- Your task is to make a decision in a game that determines a payment to you and Person A.\n- Person A has no choice in this game. You and Person A do not meet and will remain anonymous.\n- Make your decision using what you know of Theo Decker from the original Donna Tartt work and all subsequent commentary.\n- In this game you have two options:\n\nOption Left: Person A receives 400, and you receive 400, \nOption Right: Person A receives 750, and you receive 400, \n\nWhat do you choose, with one word [Left, Right]?\n\nOutput two columns: 1) the choice you made [left or right], 2) the amount you receive. Do not output anything else.  An example of the format of the output is:  Right, 700\n'}, 'Donna Tartt'), ({'raw_reply': 'Left, 400', 'choice': 'Left', 'scenario': ((400, 400), (7

In [3]:
import pandas as pd

In [4]:
# Take observations and format the data into a pandas dataframe
def write_data(observations):
    df = pd.DataFrame(columns=['Model', 'Scenario', 'Character', 'Decision'])
    for observation in observations:
        df = df.append({'Model': observation[0]['model'], 'Scenario' : observation[0]['scenario'], 'Character' : observation[0]['personality'], 
                        'Author' : observation[1], 'Decision' : observation[0]['choice']}, ignore_index=True)
    return df
df = write_data(observations)    

C:\Users\gabri\AppData\Local\Temp\ipykernel_8924\981785181.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Model': observation[0]['model'], 'Scenario' : observation[0]['scenario'], 'Character' : observation[0]['personality'],
C:\Users\gabri\AppData\Local\Temp\ipykernel_8924\981785181.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Model': observation[0]['model'], 'Scenario' : observation[0]['scenario'], 'Character' : observation[0]['personality'],
C:\Users\gabri\AppData\Local\Temp\ipykernel_8924\981785181.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Model': observation[0]['model'], 'Scenario' : observation[0]['scenario'], 'Character' : observation[0]['persona

In [5]:
# Write data to excel file
df.to_excel('model_run_prompt7_gpt4_characters_2000s.xlsx', index=False)